# Create shapes of fixed size in napari

In [10]:
import napari
import numpy as np
from skimage.data import human_mitosis, cells3d
from napari.layers import Shapes, Points

In [172]:
image = human_mitosis()
image = cells3d()[:, 1]
print(image.shape)

(60, 256, 256)


In [173]:
viewer = napari.Viewer()
viewer.add_image(image, name='image')

<Image layer 'image' at 0x24ce2a42250>

In [ ]:
from magicgui import magicgui
from napari.layers import Shapes
import napari.types
import numpy as np

@magicgui(
    call_button="Draw",
    shape_type={"choices": ["rectangle", "ellipse"]},  # Dropdown for shape type
    shape_size_x={"widget_type": "SpinBox", "min": 1, "max": 5000, "step": 1},
    shape_size_y={"widget_type": "SpinBox", "min": 1, "max": 5000, "step": 1},
)
def make_fixed_shapes_widget(
    points: napari.types.PointsData,
    shape_type: str = "rectangle",
    shape_size_x: int = 512,
    shape_size_y: int = 512,
    viewer: napari.Viewer = None,
) -> Shapes:
    """Create shapes of fixed size at points layer coordinates.
    
    Parameters
    ----------
    points : napari.types.PointsData
        Coordinates of the points layer.
    shape_type : str
        Type of shape to create. Can be 'rectangle' or 'ellipse'.
    shape_size_x : int
        Width of the shape.
    shape_size_y : int
        Height of the shape.
    viewer : napari.Viewer, optional
        Viewer instance to use for the dimensions order.
        
    Returns
    -------
    Shapes
        Shapes layer with the created shapes."""
    if points is None:
        raise ValueError("No points provided. Please select a points layer.")
    dims_order = tuple(range(points.ndim))
    if viewer is not None:
        dims_order = viewer.dims.order
    shape_size = (shape_size_y, shape_size_x)
    odd_shape = [size % 2 for size in shape_size]
    
    shapes_data = []
    for coord in points:
        shape_data = np.array([
            [coord[dims_order[-2]] - (shape_size[-2] // 2), coord[dims_order[-1]] - (shape_size[-1] // 2)],  # Top-left
            [coord[dims_order[-2]] - (shape_size[-2] // 2), coord[dims_order[-1]] + (shape_size[-1] // 2) + odd_shape[-1]],  # Bottom-left
            [coord[dims_order[-2]] + (shape_size[-2] // 2) + odd_shape[-2], coord[dims_order[-1]] + (shape_size[-1] // 2) + odd_shape[-1]],  # Bottom-right
            [coord[dims_order[-2]] + (shape_size[-2] // 2) + odd_shape[-2], coord[dims_order[-1]] - (shape_size[-1] // 2)],  # Top-right
        ])
        # Insert extra coordinates for higher dimensions
        # For example, if the shape is 3D, we need to add the z-coordinates
        extra_coords = np.take(coord, indices=dims_order[:-2], axis=0)
        for i, ec in enumerate(extra_coords):
            shape_data = np.insert(shape_data, 0, round(ec), axis=-1)
        shape_data = shape_data[:, np.argsort(dims_order)]
        shapes_data.append(shape_data)
    
    return Shapes(
        data=shapes_data,
        shape_type=[shape_type for _ in points],
    )

In [177]:
viewer.window.add_dock_widget(
    make_fixed_shapes_widget,
    area='right',
    name='Create Fixed Shapes',
)

In [160]:
viewer.dims.order

(2, 0, 1)

In [156]:
np.argsort(viewer.dims.order)

array([1, 2, 0])

In [157]:
array =viewer.layers[-1].data[0]
array

array([53.56605936, 29.        , 89.68663588])

[53.56605936 29.         89.68663588]
['x', 'z', 'y']
[89.68663588]
[[43.56605936  4.        ]
 [43.56605936 55.        ]
 [63.56605936 55.        ]
 [63.56605936  4.        ]]
new_shape data [[89.68663588 43.56605936  4.        ]
 [89.68663588 43.56605936 55.        ]
 [89.68663588 63.56605936 55.        ]
 [89.68663588 63.56605936  4.        ]]
[24.         91.74852958 53.56498319]
['x', 'z', 'y']
[53.56498319]
[[ 14.          66.74852958]
 [ 14.         117.74852958]
 [ 34.         117.74852958]
 [ 34.          66.74852958]]
new_shape data [[ 53.56498319  14.          66.74852958]
 [ 53.56498319  14.         117.74852958]
 [ 53.56498319  34.         117.74852958]
 [ 53.56498319  34.          66.74852958]]
[ 33.45284967 223.81009449 255.        ]
['x', 'z', 'y']
[255.]
[[ 23.45284967 198.81009449]
 [ 23.45284967 249.81009449]
 [ 43.45284967 249.81009449]
 [ 43.45284967 198.81009449]]
new_shape data [[255.          23.45284967 198.81009449]
 [255.          23.45284967 249.81009449]
 [

In [147]:
array.shape

(4, 3)

In [152]:
array[:, np.argsort(viewer.dims.order)]

array([[28.56606, 29.     , 79.68664],
       [79.56606, 29.     , 79.68664],
       [79.56606, 29.     , 99.68664],
       [28.56606, 29.     , 99.68664]], dtype=float32)

In [29]:
tuple(range(viewer.layers[1].ndim))

(0, 1, 2)

In [30]:
7 //2

3

In [31]:
odd_shape = [size % 2 for size in (51, 50)]

In [35]:
odd_shape[-2]

1

In [41]:
array = viewer.layers[-1].data[0]

In [55]:
array

array([[ 9.       ,  6.9195333],
       [ 9.       , 57.919533 ],
       [49.       , 57.919533 ],
       [49.       ,  6.9195333]], dtype=float32)

In [65]:
# insert new column in array
new_array = np.insert(array, 0, 0, axis=-1)
new_array

array([[ 0.       ,  9.       ,  6.9195333],
       [ 0.       ,  9.       , 57.919533 ],
       [ 0.       , 49.       , 57.919533 ],
       [ 0.       , 49.       ,  6.9195333]], dtype=float32)

In [60]:
new_array = np.expand_dims(array, axis=0)
new_array.shape

(1, 4, 2)

In [63]:
new_array

array([[[ 9.       ,  6.9195333],
        [ 9.       , 57.919533 ],
        [49.       , 57.919533 ],
        [49.       ,  6.9195333]]], dtype=float32)

In [61]:
extra_coords = viewer.layers[-2].data[0][:-2]

In [70]:
viewer.dims.order

(0, 1, 2)

In [78]:
np.take(viewer.layers[-2].data[0], indices=viewer.dims.order[:-3], axis=0)

array([], dtype=float64)

In [67]:
extra_coords[0]

np.float64(29.0)

In [62]:
# Insert extra coords in new axis in array
new_array_B = np.insert(new_array, 0, extra_coords, axis=0)
new_array_B

array([[[29.       , 29.       ],
        [29.       , 29.       ],
        [29.       , 29.       ],
        [29.       , 29.       ]],

       [[ 9.       ,  6.9195333],
        [ 9.       , 57.919533 ],
        [49.       , 57.919533 ],
        [49.       ,  6.9195333]]], dtype=float32)

In [50]:
viewer.layers[-2].data[0][:-2]

array([29.])

In [48]:
np.array([None, 1, 2])

array([None, 1, 2], dtype=object)